In [ ]:
# !pip install PIMS
# !pip install trackpy
# !pip install pandas==1.5.3
# !pip install opencv-python
# !pip install plotly

In [ ]:
# change the following to %matplotlib notebook for interactive plotting
%matplotlib inline

import numpy as np
import pandas as pd

import pims
import trackpy as tp
import os

import cv2

import matplotlib  as mpl 
import matplotlib.pyplot as plt 
import plotly.express as px

# Optionally, tweak styles.
mpl.rc('figure',  figsize=(15, 12)) #(10, 6)
mpl.rc('image', cmap='gray')

In [ ]:
# to read standard files, including tiff stacks, we can use pims.open
my_img = pims.open('tiff_series_processed/*.tif')

In [ ]:
my_img

In [ ]:
my_img.shape

In [ ]:
from stardist.models import StarDist2D, Config2D
from stardist.data import test_image_nuclei_2d
from stardist.plot import render_label
from csbdeep.utils import normalize
import matplotlib.pyplot as plt

In [ ]:
# prints a list of available models
StarDist2D.from_pretrained()

In [ ]:
# define a pretrained model to segment nuclei in fluorescence images (download from pretrained)
model = StarDist2D.from_pretrained('2D_versatile_fluo') # 2D_versatile_he

In [ ]:
plt.figure(figsize=(9, 6))
plt.imshow(my_img[0], cmap='gray')
plt.axis('off')

In [ ]:
@pims.pipeline
def stardist_segm(img):
#     img = cv2.resize(img, None, fx = 2, fy = 2)
    img_labels, img_details = model.predict_instances(normalize(img, 1.0, 99.8), prob_thresh=0.71, nms_thresh=0.8)
    return img_labels

In [ ]:
label_image = stardist_segm(my_img)

In [ ]:
label_image

In [ ]:
img_num = 9

plt.subplot(1, 2, 1)
plt.imshow(my_img[img_num], cmap='gray')
plt.axis('off')
plt.title('input image')

plt.subplot(1, 2, 2)
plt.imshow(render_label(label_image[img_num], img=my_img[img_num]))
# plt.imshow(he_labels)
plt.axis('off')
plt.title('prediction + input overlay')

In [ ]:
import skimage

In [ ]:
%%time
features = pd.DataFrame()

for num, img in enumerate(my_img):
    for region in skimage.measure.regionprops(label_image[num], intensity_image=img):
        # store fuatures
        dataset = pd.DataFrame({
            'y': [region.centroid[0]],
            'x': [region.centroid[1]],
            'frame': [num],
            'area': [region.area],
            'brightness': [region.intensity_mean],
        })
        
        features = pd.concat([features, dataset])
        
#         features = features.append([{'y': region.centroid[0],
#                                     'x': region.centroid[1],
#                                     'frame': num,
#                                     'area': region.area,
#                                     },])

In [ ]:
features.head

In [ ]:
features.to_csv('data_out/feb20_S32.csv.zip', compression='gzip', index=False)

In [ ]:
# read 'features' from file 

# features = pd.read_csv('data_out/feb20_S32.csv.zip', compression='gzip')
# features.head

In [ ]:
# create copy of features to make modifications
features_modif = features.copy()

In [ ]:
tp.annotate(features_modif[features_modif.frame==(8)], my_img[8], plot_style={'markersize':2});


In [ ]:
# plot object areas to investigate the distribution. You can further filter out objects based on size or intensity. Not

fig = px.histogram(features_modif['area'], nbins=20, template="simple_white")

fig.show()

In [ ]:
# filter out by area (make the histogram looks more or less normal distrubution)

features_modif = features_modif.loc[(features_modif['area'] > 40) & (features_modif['area'] < 419)]

In [ ]:
# plot new histogram

fig = px.histogram(features_modif['area'], nbins=20, template="simple_white")

fig.show()

In [ ]:
# plot the brightness (called as 'mass' in trackpy)

fig = px.histogram(features_modif['brightness'], nbins=20, template="simple_white")

fig.show()

In [ ]:
# filter out by intensity, then give it the name feature as it was originally

features_modif = features_modif.loc[(features_modif['brightness'] > 90)]

In [ ]:
# plot the brightness (called as 'mass' in trackpy)

fig = px.histogram(features_modif['brightness'], nbins=20, template="simple_white")

fig.show()

In [ ]:
tp.annotate(features_modif[features_modif.frame==(3)], my_img[3], plot_style={'markersize':2});

In [ ]:
# Bubble tracking
# we must specify a maximum displacement, the farthest an object can travel between frames (search_range)
# We allow for the possibility that an object might be missed for a few frames and then
# Memory keeps track of disappeared objects and maintains their ID for up to some number
# Here we use 5 frames.
# Note that the term particle refers to an object of interest.

search_range = 10
t = tp.link_df(features_modif, search_range, memory=5) # memory=5
t.head

In [ ]:
img.shape

In [ ]:
(img.shape[1], img.shape[0])

In [ ]:
# plot trajectories

overlay = img.copy()
cv2.rectangle(overlay, (0, 0), (img.shape[1]-2, img.shape[0]), (255, 255, 255), -1)  # A filled rectangle
alpha = 0.7  # Transparency factor.
img_bkgr = cv2.addWeighted(overlay, alpha, img, 1-alpha, 0)

tp.plot_traj(t, superimpose=img_bkgr, plot_style={'linewidth': 1.8}) # remove the superimpose part to just plot the trajectories

In [ ]:
# is there an overall drift? If so, we need to correct for it
drift = tp.compute_drift(t)

In [ ]:
drift.plot(figsize=(10, 6)) # if there is a horizontal line, there is no drift
plt.show()

In [ ]:
# correct drift
t_corrected = tp.subtract_drift(t.copy(), drift)

In [ ]:
# plot drift plots one more to be sure it works well
drift = tp.compute_drift(t_corrected)

drift.plot(figsize=(10, 6)) # if there is a horizontal line, there is no drift
plt.show()

In [ ]:
overlay = img.copy()
cv2.rectangle(overlay, (0, 0), (img.shape[1]-2, img.shape[0]), (255, 255, 255), -1)  # A filled rectangle
alpha = 0.7  # Transparency factor.
img_bkgr = cv2.addWeighted(overlay, alpha, img, 1-alpha, 0)

ax = tp.plot_traj(t_corrected, superimpose=img_bkgr, plot_style={'linewidth': 1.8})
plt.show()

In [ ]:
t_corrected.to_csv('data_out/feb20_S32_t_corrected.csv', index=False)

In [ ]:
# compute the mean squared displacement (MSD) of each particle using the imsd function, and plot MSD vs. lag time
# im = tp.imsd(t_corrected, 0.11073, 62.2, max_lagtime=622) # microns per pixel = 0.3069 (from Nikon reader (footer of the main window)), frames per second = 1.0 (1 frame per about 1 sec)

In [ ]:
# fig, ax = plt.subplots(figsize=(10, 6))
# ax.plot(im.index, im, 'k-', alpha=0.1) # black lines, semitransparent
# ax.set(ylabel=r'$\langle \Delta r^2 \rangle$ [$\mu$m$^2$]',
#       xlabel='lag time $t$')
# ax.set_xscale('log')
# ax.set_yscale('log')

In [ ]:
# Ensemble Mean Squared Displacement

# !!! This doesn't work for the latest pandas version (2.0.3). Downgrade version to 1.5.3 (pip install pandas==1.5.3)

em = tp.emsd(t_corrected, 0.10358, 120., max_lagtime=6000) # 0.06905
# em = tp.emsd(t_corrected, 0.11073, 62.2, max_lagtime=622)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
# ax.plot(im.index, im, 'o', color='red')
ax.plot(em.index, em, 'o')
ax.set(ylabel=r'$\langle \Delta r^2 \rangle$ [$\mu$m$^2$]',
      xlabel='lag time $t$')
# ax.set_ylim(0.1, 100)
ax.set_xscale('log')
ax.set_yscale('log')

In [ ]:
em.head

In [ ]:
# write MSD in file

import csv

with open('data_out/feb20_S32_EMSD.csv', 'w') as csvfile:
    writer = csv.writer(csvfile)#, delimiter='\t')
    writer.writerow(["lag time t", "dr^2"])
    writer.writerows(zip(em.index, em))

In [ ]:
# Fit this ensemble mean-squared displacement to a power law. Linear regression in log space

# plt.figure(figsize=(10, 6))
# plt.ylabel=(r'$\langle \Delta r^2 \rangle$ [$\mu$m$^2$]')
# plt.xlabel=('lag time $t$')
# tp.utils.fit_powerlaw(em) #performs linear best fit in log space, plots

In [ ]:
# test how dataframe 't_corrected' was written to file 

# f_corr = pd.read_csv('data_out/BE4-2D_4000frames/BE4-2D_t_corrected.csv')
# f_corr.head

In [ ]:
# NOT corrected Ensemble Mean Squared Displacement

# em_notcor = tp.emsd(t, 0.3069, 1.0, max_lagtime=4000)

In [ ]:
# fig, ax = plt.subplots(figsize=(10, 6))
# ax.plot(em_notcor.index, em_notcor, 'o')
# ax.set(ylabel=r'$\langle \Delta r^2 \rangle$ [$\mu$m$^2$]',
#       xlabel='lag time $t$')
# # ax.set_ylim(0.1, 100)
# ax.set_xscale('log')
# ax.set_yscale('log')